In [3]:
import pandas as pd
import numpy as np
import csv
import os

from tqdm import tqdm_notebook as tqdm
import scipy.ndimage
import pydicom
import datetime

In [ ]:
def load_scan(PATH):
    # try:
    #     slices = [pydicom.dcmread(os.path.join(PATH,s)) for s in os.listdir(PATH) if s.endswith('.dcm')]
    # except pydicom.errors.InvalidDicomError:
    #     slices = [pydicom.dcmread(os.path.join(PATH,s), force=True) for s in os.listdir(PATH) if s.endswith('.dcm')]
    slices = [pydicom.dcmread(os.path.join(PATH,s)) for s in os.listdir(PATH) if s.endswith('.dcm')]
    slices.sort(key = lambda x: -float(x.ImagePositionPatient[2]))               #check - or +!
    
    thick1 = np.abs(slices[1].ImagePositionPatient[2] - slices[2].ImagePositionPatient[2])
    thick2 = np.abs(slices[20].ImagePositionPatient[2] - slices[21].ImagePositionPatient[2])
    assert abs(thick1 - thick2)<0.1

    try:
        slice_thickness = np.abs(slices[1].ImagePositionPatient[2] - slices[2].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[1].SliceLocation - slices[2].SliceLocation)
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices


def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])
    # Convert to int16
    image = image.astype(np.int16)
    # Set outside-of-scan pixels to 0
    # The intercept is usually -1024, so air is approxiamately 0
    image[image == -2000] = 0
    # Convert to Hounsfield units (HU)
    for i in range(len(slices)):
        intercept = slices[i].RescaleIntercept
        slope = slices[i].RescaleSlope
        if slope != 1:
            image[i] = slope * image[i].astype(np.float64)
            image[i] = image[i].astype(np.int16)
        image[i] += np.int16(intercept)
    return np.array(image, dtype=np.int16)


def resize(voxel, spacing, new_spacing=[1.0,1.0,1.0]):
    '''Resize `voxel` from `spacing` to `new_spacing`.'''
    resize_factor = np.ones_like(voxel.shape).astype(float)
    resize_factor = spacing / new_spacing
    return scipy.ndimage.interpolation.zoom(voxel, resize_factor, mode='nearest')
    #return voxel


def new_window_map(v):
    '''Use lung windown to map CT voxel to grey.'''
    window_low = -1000.
    window_high = 400.
    assert v.min() <= window_low
    return np.round(np.clip((v - window_low) / (window_high - window_low) * 255., 0, 255)).astype(np.int16)

def get_patient_info(PATH):
    slc = pydicom.dcmread(os.path.join(PATH,os.listdir(PATH)[0]))
    sex = slc.PatientSex
    age = slc.PatientAge
    date = slc.AcquisitionDate
    return sex, age, date

def get_main_info(DICOM_PATH):
    dcm_slices = load_scan(DICOM_PATH)
    image_array = get_pixels_hu(dcm_slices)
    spacing = np.array([dcm_slices[0].SliceThickness] + map(float, dcm_slices[0].PixelSpacing), dtype=np.float32)
    origin = np.array(list(reversed(map(float, dcm_slices[0].ImagePositionPatient))))
    direction = np.array(dcm_slices[0].ImageOrientationPatient)
    return image_array, spacing, origin, direction

def get_info(DICOM_PATH):
    dcm_slices = load_scan(DICOM_PATH)
    image_array = get_pixels_hu(dcm_slices)
    spacing = np.array([dcm_slices[0].SliceThickness] + map(float, dcm_slices[0].PixelSpacing), dtype=np.float32)
    origin = np.array(list(reversed(map(float, dcm_slices[0].ImagePositionPatient))))
    direction = np.array(dcm_slices[0].ImageOrientationPatient)
    resize_array = resize(image_array,spacing)
    voxel = new_window_map(resize_array)
    return image_array, spacing, origin, direction, resize_array, voxel